In [ ]:
import pandas as pd
import pyomo.environ as pyo
import numpy as np
from collections import namedtuple
import yaml
from functools import reduce

Parameters = namedtuple("Parameters", ["D", "M", "I", "A", "B"])
Result = namedtuple("Result", ["cons", "charge", "discharge", "fitness"])

In [ ]:
N = 24
D = 1.0

COST_TYPES = ["co2", "price"]
AREAS = ["DK1", "DK2", "DE"]
CAPACITIES = [0.0, 0.5, 1.0, 2.0]

start_time = pd.Timestamp("2019-01-01 00:00").tz_localize("Europe/Copenhagen")
end_time = pd.Timestamp("2020-01-01 00:00").tz_localize("Europe/Copenhagen")

## Parameters

In [ ]:
def optimize_cost(costs, params, solver):
    assert(len(costs) == N)

    model = pyo.ConcreteModel()
    model.x = pyo.Var(range(N), domain=pyo.NonNegativeReals)
    model.y = pyo.Var(range(N), domain=pyo.NonNegativeReals)
    model.z = pyo.Var(range(N), domain=pyo.NonNegativeReals)

    model.objective = pyo.Objective(
        expr=sum(costs[i] * (model.x[i] + model.y[i]) for i in range(N)),
        sense=pyo.minimize
    )

    model.constraints = pyo.ConstraintList()

    # Cover energy demand
    for i in range(N):
        model.constraints.add(model.x[i] + model.z[i] == params.D)

    # Cap charge and discharge rates
    for i in range(N):
        model.constraints.add(model.y[i] <= params.A)
        model.constraints.add(model.z[i] <= params.B)

    # Cap battery above zero
    for j in range(1, N):
        model.constraints.add(sum(model.y[i] - model.z[i] for i in range(j)) + I >= 0.0)

    # Cap battery at capacity
    for j in range(1, N):
        model.constraints.add(sum(model.y[i] - model.z[i] for i in range(j)) + I <= params.M)

    # Ensure battery is at initial capacity at end of week
    model.constraints.add(sum(model.y[i] - model.z[i] for i in range(N)) == 0.0)

    solution = solver.solve(model)
    xs = np.array(list(model.x[i].value for i in range(N)))
    ys = np.array(list(model.y[i].value for i in range(N)))
    zs = np.array(list(model.z[i].value for i in range(N)))

    fitness = sum(costs * (xs + ys))

    return Result(xs, ys, zs, fitness)

In [ ]:
def optimize_res(res, params, solver):
    assert(len(res) == N)

    model = pyo.ConcreteModel()
    model.x = pyo.Var(range(N), domain=pyo.NonNegativeReals)
    model.y = pyo.Var(range(N), domain=pyo.NonNegativeReals)
    model.z = pyo.Var(range(N), domain=pyo.NonNegativeReals)
    model.u = pyo.Var(range(N), domain=pyo.NonNegativeReals)

    model.objective = pyo.Objective(
        expr=sum(model.u[i] for i in range(N)),
        sense=pyo.maximize
    )

    model.constraints = pyo.ConstraintList()

    # Cover energy demand
    for i in range(N):
        model.constraints.add(model.x[i] + model.z[i] == params.D)

    # Cap charge and discharge rates
    for i in range(N):
        model.constraints.add(model.y[i] <= params.A)
        model.constraints.add(model.z[i] <= params.B)

    # Cap battery above zero
    for j in range(1, N):
        model.constraints.add(sum(model.y[i] - model.z[i] for i in range(j)) + I >= 0.0)

    # Cap battery at capacity
    for j in range(1, N):
        model.constraints.add(sum(model.y[i] - model.z[i] for i in range(j)) + I <= params.M)

    # Ensure battery is at initial capacity at end of week
    model.constraints.add(sum(model.y[i] - model.z[i] for i in range(N)) == 0.0)
    
    # Cap u_i below x_i and r_i
    for i in range(N):
        model.constraints.add(model.u[i] <= res[i])
        model.constraints.add(model.u[i]  <= model.x[i] + model.y[i])

    solution = solver.solve(model)
    xs = np.array(list(model.x[i].value for i in range(N)))
    ys = np.array(list(model.y[i].value for i in range(N)))
    zs = np.array(list(model.z[i].value for i in range(N)))
    us = np.array(list(model.u[i].value for i in range(N)))

    fitness = sum(us)

    return Result(xs, ys, zs, fitness)

In [ ]:
def optimize_res_and_cost(res, costs, params, solver):
    assert(len(costs) == N)
    assert(len(res) == N)
    
    model = pyo.ConcreteModel()
    model.x = pyo.Var(range(N), domain=pyo.NonNegativeReals)
    model.y = pyo.Var(range(N), domain=pyo.NonNegativeReals)
    model.z = pyo.Var(range(N), domain=pyo.NonNegativeReals)
    model.u = pyo.Var(range(N), domain=pyo.NonNegativeReals)

    model.objective = pyo.Objective(
        expr=sum(costs[i] * model.u[i] for i in range(N)),
        sense=pyo.minimize
    )

    model.constraints = pyo.ConstraintList()

    # Cover energy demand
    for i in range(N):
        model.constraints.add(model.x[i] + model.z[i] == params.D)

    # Cap charge and discharge rates
    for i in range(N):
        model.constraints.add(model.y[i] <= params.A)
        model.constraints.add(model.z[i] <= params.B)

    # Cap battery above zero
    for j in range(1, N):
        model.constraints.add(sum(model.y[i] - model.z[i] for i in range(j)) + I >= 0.0)

    # Cap battery at capacity
    for j in range(1, N):
        model.constraints.add(sum(model.y[i] - model.z[i] for i in range(j)) + I <= params.M)

    # Ensure battery is at initial capacity at end of week
    model.constraints.add(sum(model.y[i] - model.z[i] for i in range(N)) == 0.0)
    
    # Set u[i] >= x[i] + y[i] - res[i]
    for i in range(N):
        model.constraints.add(model.u[i] >= model.x[i] + model.y[i] - res[i])

    solution = solver.solve(model)
    xs = np.array(list(model.x[i].value for i in range(N)))
    ys = np.array(list(model.y[i].value for i in range(N)))
    zs = np.array(list(model.z[i].value for i in range(N)))
    us = np.array(list(model.u[i].value for i in range(N)))

    fitness = sum(costs * us)

    return Result(xs, ys, zs, fitness)

In [ ]:
def optimize_series_cost(data, costs_col, params, solver):
    ndays = len(data) // 24
    output_parts = []
    for day in range(ndays):
        subdata = data.iloc[day*24: (day+1)*24]
        costs = subdata[costs_col].values
        result = optimize_cost(costs, params, solver)
        output_parts.append(
            pd.DataFrame(
                dict(time=subdata.time, cons=result.cons, charge=result.charge, discharge=result.discharge)
            )
        )
    
    output = pd.concat(output_parts, axis=0)
    return output

In [ ]:
def optimize_series_res(data, res_col, params, solver):
    ndays = len(data) // 24
    output_parts = []
    
    resnorm = data[res_col] / data[res_col].sum() * len(data) * D
    data = data.assign(**{res_col: resnorm})
    
    for day in range(ndays):
        subdata = data.iloc[day*24: (day+1)*24]
        res = subdata[res_col].values
        result = optimize_res(res, params, solver)
        output_parts.append(
            pd.DataFrame(
                dict(time=subdata.time, cons=result.cons, charge=result.charge, discharge=result.discharge)
            )
        )
    
    output = pd.concat(output_parts, axis=0)
    return output

In [ ]:
def optimize_series_res_and_cost(data, res_col, cost_col, params, solver):
    ndays = len(data) // 24
    output_parts = []
    
    resnorm = data[res_col] / data[res_col].sum() * len(data) * D
    data = data.assign(**{res_col: resnorm})
    
    for day in range(ndays):
        subdata = data.iloc[day*24: (day+1)*24]
        res = subdata[res_col].values
        costs = subdata[cost_col].values

        result = optimize_res_and_cost(res, costs, params, solver)
        output_parts.append(
            pd.DataFrame(
                dict(time=subdata.time, cons=result.cons, charge=result.charge, discharge=result.discharge)
            )
        )
    
    output = pd.concat(output_parts, axis=0)
    return output

In [ ]:
def read_portfolio(portfolio) -> pd.DataFrame:
    data = []
    for site in portfolio["sites"]:
        path = f"data/sites/{site}.csv"
        df = (
            pd.read_csv(
                path,
                comment="#",
                usecols=["time", "electricity"]
            )
            .assign(time=lambda df: pd.to_datetime(df.time, format="%Y-%m-%d %H:%M", utc=True).dt.tz_convert("Europe/Berlin"))
            .query("time >= @start_time & time < @end_time")
            .set_index("time").resample("H")
            .mean().ffill()
        )
        
        data.append(df)
    
    output = (
        reduce(lambda a, b: a + b, data)
        .rename({"electricity": "res"}, axis=1)
        .reset_index()
    )
    return output

## Optimize for CO2 and spot price for each price area

In [ ]:
solver = pyo.SolverFactory("glpk")

In [ ]:
for cost_type in COST_TYPES:
    for area in AREAS:
        data = (
            pd.read_csv(f"data/{cost_type}/{area}.csv")
            .assign(time=lambda df: pd.to_datetime(df.time, utc=True).dt.tz_convert("Europe/Copenhagen"))
            .assign(**{cost_type: lambda df: df[cost_type].clip(0)})
            .query("time >= @start_time & time < @end_time")
        )

        for M in CAPACITIES:
            I, A, B = M * 0.5, M * 0.5, M * 0.5
            params = Parameters(D=D, M=M, I=I, A=A, B=B) 

            results = (
                optimize_series_cost(data, cost_type, params, solver)
                .assign(total_energy=lambda df: df.cons + df.charge)
            )
            results.to_csv(f"data/schedules/{cost_type}_{area}_M={M}.csv", index=False)

## Optimize for RES portfolio utilization

In [ ]:
with open("data/portfolios.yml", "r") as fp:
    portfolios = yaml.load(fp, Loader=yaml.SafeLoader)

In [ ]:
for portfolio in portfolios:
    data = read_portfolio(portfolio)
    
    for M in CAPACITIES:
        I, A, B = M * 0.5, M * 0.5, M * 0.5
        params = Parameters(D=D, M=M, I=I, A=A, B=B)
        
        results = (
            optimize_series_res(data, "res", params, solver)
            .assign(total_energy=lambda df: df.cons + df.charge)
        )
        results.to_csv(f"data/schedules/res_{portfolio['name']}_M={M}.csv", index=False)

## Optimize for RES portfolio and cost

In [ ]:
for portfolio in portfolios:
    res_data = read_portfolio(portfolio)
    
    for cost_type in COST_TYPES:
        cost_data = (
            pd.read_csv(f"data/{cost_type}/{portfolio['area']}.csv")
            .assign(time=lambda df: pd.to_datetime(df.time, utc=True).dt.tz_convert("Europe/Copenhagen"))
            .assign(**{cost_type: lambda df: df[cost_type].clip(0)})
            .query("time >= @start_time & time < @end_time")
        )
        
        data = pd.merge(res_data[["time", "res"]], cost_data[["time", cost_type]], on="time", how="inner")

        for M in CAPACITIES:
            I, A, B = M * 0.5, M * 0.5, M * 0.5
            params = Parameters(D=D, M=M, I=I, A=A, B=B)
            
            results = (
                optimize_series_res_and_cost(data, "res", cost_type, params, solver)
                .assign(total_energy=lambda df: df.cons + df.charge)
            )
            results.to_csv(f"data/schedules/res_and_{cost_type}_{portfolio['name']}_M={M}.csv", index=False)

# Aggregate results

In [ ]:
rows = []

for portfolio in portfolios:
    res = (
        read_portfolio(portfolio)
        .assign(time=lambda df: pd.to_datetime(df.time, utc=True).dt.tz_convert("Europe/Copenhagen"))
    )
    
    cost_co2 = (
        pd.read_csv(f"data/co2/{portfolio['area']}.csv")
        .assign(
            time=lambda df: pd.to_datetime(df.time, utc=True).dt.tz_convert("Europe/Copenhagen"),
            co2=lambda df: df.co2.clip(0)
        )
    )

    cost_price = (
        pd.read_csv(f"data/price/{portfolio['area']}.csv")
        .assign(
            time=lambda df: pd.to_datetime(df.time, utc=True).dt.tz_convert("Europe/Copenhagen"),
            price=lambda df: df.price.clip(0)
        )
    )

    for M in CAPACITIES:
        for cost_type in ["co2", "price", "res", "res_and_co2", "res_and_price"]:
            if cost_type in ["co2", "price"]:
                schedule_path = f"data/schedules/{cost_type}_{portfolio['area']}_M={M}.csv"
            else:
                schedule_path = f"data/schedules/{cost_type}_{portfolio['name']}_M={M}.csv"
            
            schedule = (
                pd.read_csv(schedule_path)
                .assign(time=lambda df: pd.to_datetime(df.time, utc=True).dt.tz_convert("Europe/Copenhagen"))
            )

            data = (
                schedule[["time", "total_energy"]].rename({"total_energy": "consumed_energy"}, axis=1)
                .merge(cost_co2[["time", "co2"]].rename({"co2": "co2_per_kwh"}, axis=1), on="time", how="inner")
                .merge(cost_price[["time", "price"]].rename({"price": "eur_per_kwh"}, axis=1), on="time", how="inner")
                .merge(res.rename({"res": "portfolio_energy"}, axis=1), on="time", how="inner")
            )

            data = data.assign(
                consumed_energy=lambda df: df.consumed_energy / df.consumed_energy.sum() * 1000,
                portfolio_energy=lambda df: df.portfolio_energy / df.portfolio_energy.sum() * 1000,
                co2=lambda df: df.consumed_energy * df.co2_per_kwh,
                eur=lambda df: df.consumed_energy * df.eur_per_kwh,
                portfolio_coverage=lambda df: np.minimum(df.consumed_energy, df.portfolio_energy),   
            )

            data.to_csv(f"output/battery/{portfolio['name']}_{cost_type}_M={M}.csv", index=False)
            
            flat = data.consumed_energy.mean()
            
            baseline_co2 = data.co2_per_kwh.sum() * flat
            optimized_co2 = data.co2.sum()
            improvement_co2 = (baseline_co2 - optimized_co2) / baseline_co2

            baseline_eur = data.eur_per_kwh.sum() * flat
            optimized_eur = data.eur.sum()
            improvement_eur = (baseline_eur - optimized_eur) / baseline_eur

            baseline_res = np.minimum(flat, data.portfolio_energy).sum() / 1000
            optimized_res = data.portfolio_coverage.sum() / 1000
            improvement_res = optimized_res - baseline_res
            
            rows.append({
                "Portfolio": portfolio["name"],
                "Price area": portfolio["area"],
                "Target": cost_type,
                "Battery capacity": M,
                "Baseline CO2 (g/MWh)": baseline_co2,
                "Optimized CO (g/MWh)": optimized_co2,
                "Improvement CO2 (%)": improvement_co2,
                "Baseline price (EUR/MWh)": baseline_eur,
                "Optimized price (EUR/MWh)": optimized_eur,
                "Improvement price (%)": improvement_eur,
                "Baseline RES (%)": baseline_res,
                "Optimized RES (%)": optimized_res,
                "Improvement RES (%)": improvement_res
            })

In [ ]:
summary = pd.DataFrame(rows)
summary

In [ ]:
summary.query("Portfolio == 'DE2' & Target in ['price', 'res_and_price', 'res']").pivot(index="Target", columns="Battery capacity", values="Improvement price (%)").plot.bar()

In [ ]:
TARGET_NAMES = {"co2": "CO2", "price": "Price", "res": "RES", "res_and_co2": "RES + CO2", "res_and_price": "RES + price"}

with pd.ExcelWriter("output/battery_summary.xlsx") as writer:
    for cost_type in summary.Target.unique():
        summary.query("Target == @cost_type").to_excel(writer, index=False, sheet_name=f"Optimized {TARGET_NAMES[cost_type]}")

In [ ]:
day = 0
ax = (
    pd.read_csv("output/battery/DE2_res_M=2.0.csv")
    .iloc[day*24:day*24+169]
    .assign(
        time=lambda df: pd.to_datetime(df.time, utc=True).dt.tz_convert("Europe/Copenhagen"),
    )
    .plot(x="time", y=["consumed_energy", "portfolio_energy"], figsize=(20, 8), drawstyle="steps-post")
)

In [ ]:
import matplotlib.ticker as mtick

ax = summary.query("Target == 'res' & `Battery capacity` > 0").pivot(index="Portfolio", columns="Battery capacity", values="Improvement RES (%)").plot.bar(figsize=(12, 6))
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))